In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
df = pd.read_csv('TWO_CENTURIES_OF_UM_RACES.csv')

C:\Users\Grzegorz\AppData\Local\Temp\ipykernel_12408\107228863.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('TWO_CENTURIES_OF_UM_RACES.csv')


In [3]:
df.sample(5)

,Year of event,Event dates,Event name,Event distance/length,Event number of finishers,Athlete performance,Athlete club,Athlete country,Athlete year of birth,Athlete gender,Athlete age category,Athlete average speed,Athlete ID
3123495,2021,30.-31.10.2021,Shodoshima Round Ultra Run (JPN),104km,137,17:35:46 h,NaN,JPN,NaN,M,NaN,5.91,307274
5682069,2013,01.06.2013,Rainier to Ruston Rail-Trail 50 Mile Ultra (USA),50mi,60,10:22:14 h,"*Puyallup, WA",USA,1963.0,M,M45,7.759,1392321
2619366,2020,16.02.2020,Borneo Dragon Back 70km (MAS),70km,39,15:12:38 h,NaN,MAS,NaN,M,NaN,4.602,827074
3569784,2022,12.11.2022,EicUt 65 (GER),65km,49,7:27:16 h,TV Haibach,GER,1957.0,M,M65,8.72,39289
2161999,2019,09.06.2019,Trail de Guerlédan (FRA),63km,988,8:27:27 h,Team Durrmann,FRA,1981.0,M,M35,7.449,192878


TODO: 

<s>Get month from event date for seasonal analysis </s>

Get country from event name for geolocation analysis

<s>Calculate athelete age at the time of the event</s>

Remove outliers from averege speed

Change athelete perfmormance dtype into something usable

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7461195 entries, 0 to 7461194
Data columns (total 13 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   Year of event              int64  
 1   Event dates                object 
 2   Event name                 object 
 3   Event distance/length      object 
 4   Event number of finishers  int64  
 5   Athlete performance        object 
 6   Athlete club               object 
 7   Athlete country            object 
 8   Athlete year of birth      float64
 9   Athlete gender             object 
 10  Athlete age category       object 
 11  Athlete average speed      object 
 12  Athlete ID                 int64  
dtypes: float64(1), int64(3), object(9)
memory usage: 740.0+ MB


## Data Preparation

In [5]:
# Removing club because it's useless
df.drop(['Athlete club'],axis=1,inplace=True)
# Removing age category, because it's redundant when we can get age column
df.drop(['Athlete age category'],axis=1, inplace=True)

In [6]:
pd.set_option('display.max_rows', None)
df['Year of event'].value_counts()

Year of event
2019    732207
2018    661483
2017    604638
2016    540073
2015    480722
2022    471598
2014    417613
2021    375604
2013    348829
2012    298981
2011    236397
2020    224203
2010    207315
2009    169969
2008    136333
2007    123766
2006    105639
2005     94253
2004     91813
2003     84114
2000     76563
2002     69190
2001     66071
1999     58672
1998     52782
1997     48538
1996     46962
1991     43954
1995     42503
1993     40793
1994     38906
1990     38775
1992     38603
1989     38491
1988     37351
1986     33014
1987     32301
1984     30692
1985     29417
1983     27542
1982     24847
1981     22577
1980     18948
1979     15790
1978     14562
1977     12781
1976      9791
1975      8498
1974      6447
1973      5938
1972      4693
1971      3984
1970      2674
1969      2213
1968      1756
1967      1359
1966      1109
1965       984
1964       830
1963       513
1962       434
1961       334
1960       270
1892       226
1959       196
1956       

In [7]:
# Rows with year of event < 1970 will be dropped, because there's very few of them
print(f"Before dropping {df[df['Year of event']<1970].shape}")
df = df.drop(df[df['Year of event'] < 1970].index)
print(f"After dropping {df[df['Year of event']<1970].shape}")

Before dropping (12995, 11)
After dropping (0, 11)


In [8]:
df['Event distance/length'].value_counts()

Event distance/length
50km                   1521926
100km                   946201
50mi                    351832
56km                    333250
87km                    210645
89km                    187987
24h                     182997
100mi                   173787
60km                    159494
45km                    157799
55km                    131047
6h                      125834
12h                     114317
80km                     97962
52km                     92611
65km                     90525
70km                     82684
53km                     71828
75km                     62811
48km                     61596
46km                     60038
90km                     57381
63km                     56926
51km                     52648
72km                     52367
54km                     51406
47km                     41177
58km                     39100
67km                     37387
57km                     37034
68km                     36551
73km             

In [9]:
# Taking only the top 100 races by number of occurances
df = df[df['Event distance/length'].isin(df['Event distance/length'].value_counts().index[:100])]

In [10]:
df['Event distance/length'].value_counts()

Event distance/length
50km              1521926
100km              946201
50mi               351832
56km               333250
87km               210645
89km               187987
24h                182997
100mi              173787
60km               159494
45km               157799
55km               131047
6h                 125834
12h                114317
80km                97962
52km                92611
65km                90525
70km                82684
53km                71828
75km                62811
48km                61596
46km                60038
90km                57381
63km                56926
51km                52648
72km                52367
54km                51406
47km                41177
58km                39100
67km                37387
57km                37034
68km                36551
73km                35643
62km                34001
78km                31891
110km               30366
40mi                30179
101km               30098
71.5km          

In [11]:
df.isnull().sum()

Year of event                     0
Event dates                       0
Event name                        0
Event distance/length             0
Event number of finishers         0
Athlete performance               2
Athlete country                   3
Athlete year of birth        526640
Athlete gender                    7
Athlete average speed             4
Athlete ID                        0
dtype: int64

In [12]:
# Percentage of missing values
(df.isnull().mean() * 100).round(2)

Year of event                0.00
Event dates                  0.00
Event name                   0.00
Event distance/length        0.00
Event number of finishers    0.00
Athlete performance          0.00
Athlete country              0.00
Athlete year of birth        7.83
Athlete gender               0.00
Athlete average speed        0.00
Athlete ID                   0.00
dtype: float64

In [13]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Use iterative imputer to predict missing values
imputer = IterativeImputer()
df['Athlete year of birth'] = imputer.fit_transform(df[['Athlete year of birth']])

In [14]:
df['Athlete year of birth'] = df['Athlete year of birth'].astype(int)

In [15]:
df['Athlete age'] = df['Year of event'] - df['Athlete year of birth']

In [16]:
df.dropna(subset=['Athlete performance','Athlete gender','Athlete average speed','Athlete country'],inplace=True)

In [17]:
# Function to extract month from date
def extract_month(date_str):
    parts = date_str.split('.')
    #print(f"Length: {len(parts)}")
    if len(parts) == 4:
        month_str = parts[2]
        month_str_clean = ''.join(filter(str.isdigit, month_str))
        return int(month_str_clean)
    if len(parts) == 3:
        month_str = parts[1]
        month_str_clean = ''.join(filter(str.isdigit, month_str))
        return int(month_str_clean)
    if len(parts) == 5:
        month_str = parts[3]
        month_str_clean = ''.join(filter(str.isdigit, month_str))
        return int(month_str_clean)
    else:
        return None


In [18]:
# Function validation
sample_dates = ["10.-11.10.2015", "02.07.2022", "28.03.-03.04.2016","31.12.2022-01.01.2023"]
for date in sample_dates:
    print(f"Date: {date} -> Month: {extract_month(date)}")

Date: 10.-11.10.2015 -> Month: 10
Date: 02.07.2022 -> Month: 7
Date: 28.03.-03.04.2016 -> Month: 4
Date: 31.12.2022-01.01.2023 -> Month: 1


In [19]:
df.isnull().sum()

Year of event                0
Event dates                  0
Event name                   0
Event distance/length        0
Event number of finishers    0
Athlete performance          0
Athlete country              0
Athlete year of birth        0
Athlete gender               0
Athlete average speed        0
Athlete ID                   0
Athlete age                  0
dtype: int64

In [20]:
# Apply the function to extract months
df['Month'] = df['Event dates'].apply(extract_month).astype(int)

In [21]:
df = df[df['Athlete average speed'].str.contains(':') == False]

In [24]:
df['Athlete average speed'] = df['Athlete average speed'].astype(float)

In [27]:
# Removing all values bigger than 13, because that's the world record for average speed of marathorn runner
df = df[df['Athlete average speed'] <= 13]

In [ ]:
mean_speed = df['Athlete average speed'].mean()
median_speed = df['Athlete average speed'].median()
std_deviation = df['Athlete average speed'].std()

# Calculate Z-scores
z_scores = df['Athlete average speed'].apply(lambda x: (x - mean_speed) / std_deviation)

# Identify outliers with absolute Z-score greater than 3 and extract 'Athlete average speed' values
outliers_speeds = df.loc[z_scores.abs() > 3, 'Athlete average speed']

print("Outliers detected using Z-Score method:")
print(outliers_speeds)

Mean: 6.933315817709238
Median: 6.841
Standard Deviation: 1.9960491310783721
Outliers detected using Z-Score method:
4663       12.955
7194        0.192
9567       12.978
9637       12.970
9638       12.933
12098      12.978
12792      12.992
12793      12.989
12794      12.970
15014      12.984
15015      12.965
15016      12.958
15017      12.928
20716      12.971
24166      12.985
24690      12.952
26276      12.975
26277      12.952
26406      12.974
26407      12.951
26559      12.951
27853      12.986
30380      12.986
31250      12.930
31339      12.943
38909      12.966
43702      12.958
47833      12.947
49678      12.977
49679      12.957
50168      12.990
58443      12.987
58444      12.941
58445      12.940
61176      12.976
63886      12.990
65153      12.938
131159     12.988
131160     12.988
131161     12.983
131162     12.967
131163     12.967
131164     12.961
131165     12.958
131166     12.956
131167     12.947
131168     12.926
131169     12.923
131170     12.923
1

In [37]:
# Remove outliers (rows with |Z| > 3)
df = df[z_scores.abs() <= 3]

C:\Users\Grzegorz\AppData\Local\Temp\ipykernel_12408\1222592710.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[z_scores.abs() <= 3]
